In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
selected_features = df[['koi_prad', 'koi_impact', 'koi_time0bk', 'koi_model_snr', 'koi_depth', 'koi_fpflag_ss', 'koi_srad', 'koi_insol', 'koi_slogg', 'koi_kepmag']]
# selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_depth', 'koi_teq', 'koi_steff']]
selected_features.head()

,koi_prad,koi_impact,koi_time0bk,koi_model_snr,koi_depth,koi_fpflag_ss,koi_srad,koi_insol,koi_slogg,koi_kepmag
0,2.83,0.586,162.513840,25.8,874.8,0,0.927,9.11,4.467,15.347
1,14.60,0.969,175.850252,76.3,10829.0,1,0.868,39.30,4.544,15.436
2,33.46,1.276,170.307565,505.6,8079.2,1,0.791,891.96,4.564,15.597
3,2.75,0.701,171.595550,40.9,603.3,0,1.046,926.16,4.438,15.509
4,2.77,0.762,172.979370,40.2,686.0,0,0.972,427.65,4.486,15.714


In [4]:
all_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
all_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


In [5]:
from sklearn.preprocessing import LabelEncoder

y = df['koi_disposition']
# X = selected_features
X = all_features

label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Originally had this in - but read that scaling is not necessarily in most RF models
# from sklearn.preprocessing import StandardScaler

# X_scaler = StandardScaler().fit(X_train)

# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)


In [14]:
rf = RandomForestClassifier(n_estimators=500)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9073226544622426

In [15]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, all_features), reverse=True)

[(0.12700112311816408, 'koi_model_snr'),
 (0.1269885891236496, 'koi_fpflag_nt'),
 (0.1259409391854445, 'koi_fpflag_co'),
 (0.09400092121818143, 'koi_fpflag_ss'),
 (0.0879815248730373, 'koi_prad'),
 (0.05247856017935128, 'koi_depth'),
 (0.043777127964173865, 'koi_fpflag_ec'),
 (0.04334097923796884, 'koi_period'),
 (0.03858732472224446, 'koi_impact'),
 (0.03446673041299793, 'koi_insol'),
 (0.032822347626586736, 'koi_teq'),
 (0.030373910256156125, 'koi_duration'),
 (0.028743886414480342, 'koi_time0bk'),
 (0.02363325704306788, 'koi_steff'),
 (0.02182904809535271, 'ra'),
 (0.020711030946304415, 'dec'),
 (0.020520017686812567, 'koi_kepmag'),
 (0.01972425074583797, 'koi_srad'),
 (0.019242894657367907, 'koi_slogg'),
 (0.007835536492819983, 'koi_tce_plnt_num')]

In [16]:
import joblib
filename = 'model_saves/rf_exoplanet.sav'
joblib.dump(rf, filename)


['model_saves/rf_exoplanet.sav']

In [17]:
# Apply the classifier to the test data
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)

accuracy_score(y_test, y_pred)

0.9073226544622426

In [ ]:
#Followed this article: https://chrisalbon.com/machine_learning/trees_and_forests/feature_selection_using_random_forest/